In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import sqlite3
import pandas as pd

In [2]:
DB_PATH = os.getenv('DB_PATH')
CSV_PATH = os.getenv('CSV_PATH')

In [3]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables['name'].tolist())

['opprc', 'secprc', 'fwdprc', 'rate']


In [11]:
# Query to get data from opprc and secprc and join them on the date column
query = """
SELECT opprc.date, opprc.exdate, opprc.best_bid, opprc.best_offer, opprc.strike_price, secprc.close
FROM opprc
JOIN secprc ON opprc.date = secprc.date
WHERE (opprc.best_offer - opprc.best_bid) / (opprc.best_bid + opprc.best_offer) < 0.001
"""
merged_df = pd.read_sql(query, conn)

print(merged_df.head())


         date      exdate  best_bid  best_offer  strike_price    close
0  2022-09-01  2022-09-16    2960.2      2965.0       1000000  3966.85
1  2022-09-01  2022-09-16    3559.0      3565.3        400000  3966.85
2  2022-09-01  2022-09-16    3159.2      3165.5        800000  3966.85
3  2022-09-01  2022-09-16    3228.2      3234.6       7200000  3966.85
4  2022-09-01  2022-09-16    3428.0      3434.4       7400000  3966.85


In [13]:
# Read the tables into pandas dataframes
opprc_df = pd.read_sql_query("SELECT date, exdate, best_bid, best_offer, strike_price FROM opprc", conn)
secprc_df = pd.read_sql_query("SELECT date, close FROM secprc", conn)

# Merge the dataframes on the 'date' column
merged_df = pd.merge(opprc_df, secprc_df, on='date').query('(best_offer - best_bid) / (best_bid + best_offer) < 0.001')

print(merged_df.head())


             date      exdate  best_bid  best_offer  strike_price    close
18992  2022-09-01  2022-09-16    2960.2      2965.0       1000000  3966.85
19207  2022-09-01  2022-09-16    3559.0      3565.3        400000  3966.85
19398  2022-09-01  2022-09-16    3159.2      3165.5        800000  3966.85
19803  2022-09-01  2022-09-16    3228.2      3234.6       7200000  3966.85
19804  2022-09-01  2022-09-16    3428.0      3434.4       7400000  3966.85
